In [ ]:
# Do some imports.
from astroquery.mast import Observations
import numpy

In [ ]:
# Let's get every time series observation available at MAST within 0.2 degrees of M67.
search_result = Observations.query_criteria(objectname="M67", dataproduct_type=["timeseries"], radius="0.2 deg")
# The return result is a Table containing the metadata you see when doing a MAST Portal search: coordinates, timestamps,
# filters, target name, mission/project, etc.
print(search_result.columns)

In [ ]:
# How many observations did we get?
print(len(search_result))

In [ ]:
# Did these all come from the K2 mission?  Answer: no!  High Level Science Products too!
print(set(search_result['project']))

In [ ]:
# Let's get a list of all the K2 IDs.  NOTE:  K2 mission-produced products always have "project" = "K2".
where_k2_mission = numpy.where(search_result['project'] == 'K2')[0]
print(search_result['target_name'][where_k2_mission][0:5])

In [ ]:
# I just want the actual ID numbers, so make a numpy array stripping away the "ktwo" prefix.
k2_ids = numpy.asarray(
        [x.strip('ktwo') for x in search_result['target_name'][where_k2_mission]]
    )
print(k2_ids)

In [ ]:
# Now we will loop over each K2 ID and download the light curves if and only if there is a LC from the mission
# AND all four of the HLSP teams.
for k2id in k2_ids:
        where_k2id = numpy.where(search_result['target_name'] == 'ktwo' + k2id)[0]
        # Does this have a POLAR light curve?
        where_polar = numpy.where(search_result['target_name'] == 'polar' + k2id)[0]
        n_polar = len(where_polar)
        # Does this have a K2SFF light curve?
        where_k2sff = numpy.where(search_result['target_name'] == 'k2sff' + k2id)[0]
        n_k2sff = len(where_k2sff)
        # Does this have an EVEREST light curve?
        where_everest = numpy.where(search_result['target_name'] == 'everest' + k2id)[0]
        n_everest = len(where_everest)
        # Does this have a K2SC light curve?
        where_k2sc = numpy.where(search_result['target_name'] == 'k2sc' + k2id)[0]
        n_k2sc = len(where_k2sc)
        if n_polar == 1 and n_k2sff == 1 and n_everest == 1 and n_k2sc == 1:
                print("K2 ID " + k2id + " - All five light curves found!")
                Observations.download_products(search_result['obsid'][where_k2id])
                Observations.download_products(search_result['obsid'][where_polar])
                Observations.download_products(search_result['obsid'][where_k2sff])
                Observations.download_products(search_result['obsid'][where_everest])
                Observations.download_products(search_result['obsid'][where_k2sc])

# Note: here I am downloading one observation at a time using the "obsid", but download_products can also accept
# a list of obsid's, or even an entire Table of data products, so I could do the download step in a single line
# if I wanted.  Note also that if you add a "curl_flag = True" to the download_products() call, it will create a
# curl shell script you can execute later, if you don't want to actually download the files right away.
                